## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

# Connect to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

Mounted at /content/drive
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=KqoV1VMboBV9-rlCpgrUF0l_AY5gv0vbSA_kdwy50sI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgT_VE7GwXVEFiIPokezcJ_TAOL3OdfXiJvsea_vDdaVRH2IvNuJaQ

Successfully saved authorization token.


In [2]:
import os
from glob import glob
!pip install shapely
!pip install geopandas
!pip install xarray
!pip install rioxarray
!pip install wxee
!pip install geojson
!pip install geemap

import geemap
import geojson
import shapely as shp
import geopandas as gpd
import wxee
import numpy as np
import matplotlib.pyplot as plt


     |████████████████████████████████| 1.0 MB 4.1 MB/s 
     |████████████████████████████████| 15.4 MB 46.1 MB/s 
     |████████████████████████████████| 6.3 MB 21.2 MB/s 
     |████████████████████████████████| 46 kB 105 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.9.0-py3-none-any.whl size=54395 sha256=363911adc567e02b6b4bd022e64aa3772a7e6c4b3de8a8d785c512de96726990
  Stored in directory: /root/.cache/pip/wheels/f2/0d/f2/29da7ed804b71eb1317c3683b373f59d7c0c4c501117f2b144
Successfully built rioxarray
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
  Created wheel for wxee: filename=wxee-0.3.2-py3-none-any.whl size=37659 sha256=b36db421bdc798a7f3fac22e25d749989eabab13537d041678bce74ffe0bcfeb
  Stored in directory: /root/.cach

## User-defined Functions

## Prepare ROI (Asia) for processing

### Load one-country parcels

In [8]:
# Load all parcels
os.chdir('/content/drive/MyDrive/THESIS_AQUAPONDS/ROI/Aquaculture_Asia_Coast_2019/Coastline_parcels_polygon')
path_parcels = glob(os.path.join(os.getcwd(), '*.geojson'))[0]
parcels = gpd.read_file(path_parcels)

In [15]:
#parcels.to_file('parcels.geojson', driver='GeoJSON')
with open('parcels.geojson') as f:
  parcels_geojson = geojson.load(f)

In [19]:
#len(parcels_geojson['features'])

266

In [9]:
# List all countries
#countries = sorted(list(set(parcels['NAME_0'])))
#print(countries)

['Bangladesh', 'Cambodia', 'China', 'India', 'Indonesia', 'Iran', 'Japan', 'Malaysia', 'Myanmar', 'North Korea', 'Pakistan', 'Philippines', 'South Korea', 'Sri Lanka', 'Taiwan', 'Thailand', 'Vietnam']


In [10]:
# Subset Parcels to one Country
#parcels_one_country = parcels[parcels['NAME_0']==countries[0]]
#parcels_one_country

,GID_0,NAME_0,geometry
208,BGD,Bangladesh,"MULTIPOLYGON (((93.22541 23.10278, 93.12052 22..."
209,BGD,Bangladesh,"MULTIPOLYGON (((93.23961 23.12663, 93.22541 23..."
252,BGD,Bangladesh,"MULTIPOLYGON (((89.31118 20.35796, 89.07910 21..."


## Processing in GEE

In [25]:
# Define roi
roi = ee.Geometry(parcels_geojson['features'][0]['geometry'])
roi.getInfo()

{'coordinates': [[[147.70812, 43.079548],
   [146.191735, 45.553622],
   [144.736087, 44.867934],
   [146.24364, 42.476689],
   [147.70812, 43.079548]]],
 'type': 'Polygon'}

In [31]:
Map = geemap.Map(center=(20, 110), zoom=3)
Map.addLayer(roi, {}, 'roi')
Map

Map(center=[20, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…